In [ ]:
import os
import pandas as pd
import shutil


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

# Set the path to the directory containing the images
images_dir = "/content/drive/My Drive/ECE 176/dataset-master/JPEGImages"

# Set the path to the CSV file containing the class labels
labels_file = "/content/drive/My Drive/ECE 176/dataset-master/bccd_labels.csv"

# Load the class labels from the CSV file into a pandas DataFrame
labels_df = pd.read_csv(labels_file)

In [ ]:
for i, row in labels_df.iterrows():

    # Get the image number and class label for this row
    image_number = row["Image"]
    class_labels = row["Category"].split(",")  # Split the class labels by comma

    if image_number<10:
      filename = f"BloodImage_0000{image_number}.jpg"
    elif image_number>9 and image_number <100:
      filename = f"BloodImage_000{image_number}.jpg"
    else:
      filename = f"BloodImage_00{image_number}.jpg"
    # Create the class folder if it doesn't exist
    # Construct the image filename based on the image number
    #filename = f"BloodImage_{image_number}.jpg"

    # Loop through each class label for this image
    for class_label in class_labels:
        # Create the class folder if it doesn't exist
        class_folder = os.path.join(images_dir, class_label.strip())  # Use strip() to remove whitespace
        os.makedirs(class_folder, exist_ok=True)

        # Move the image to the appropriate class folder
        image_path = os.path.join(images_dir, filename)
        destination_path = os.path.join(class_folder, filename)
        shutil.copy(image_path, destination_path)
        print('\nImage number:', image_number,'saved')


Image number: 0 saved

Image number: 1 saved

Image number: 2 saved

Image number: 3 saved

Image number: 4 saved

Image number: 5 saved

Image number: 6 saved

Image number: 7 saved

Image number: 8 saved

Image number: 9 saved

Image number: 10 saved

Image number: 10 saved

Image number: 11 saved

Image number: 12 saved

Image number: 13 saved

Image number: 14 saved

Image number: 15 saved

Image number: 16 saved

Image number: 17 saved

Image number: 18 saved

Image number: 19 saved

Image number: 20 saved

Image number: 21 saved

Image number: 22 saved

Image number: 23 saved

Image number: 24 saved

Image number: 26 saved

Image number: 28 saved

Image number: 29 saved

Image number: 30 saved

Image number: 31 saved

Image number: 31 saved

Image number: 32 saved

Image number: 33 saved

Image number: 34 saved

Image number: 34 saved

Image number: 35 saved

Image number: 36 saved

Image number: 37 saved

Image number: 38 saved

Image number: 39 saved

Image number: 40 saved

I

In [ ]:
import os

# Set the path to the directory containing the folders
dir_path = "/content/drive/My Drive/ECE 176/dataset-master/JPEGImages/classes"

# Loop through each folder in the directory
for folder_name in os.listdir(dir_path):
    folder_path = os.path.join(dir_path, folder_name)

    # Check if this path is a directory
    if os.path.isdir(folder_path):
        # Count the number of files in this folder
        file_count = len(os.listdir(folder_path))

        # Print the folder name and file count
        print(f"{folder_name}: {file_count}")


NEUTROPHIL: 215
BASOPHIL: 4
EOSINOPHIL: 94
MONOCYTE: 23
LYMPHOCYTE: 37


In [ ]:
import os
import numpy as np
from PIL import Image

# Set the directory of your dataset
data_dir = '/content/drive/My Drive/ECE 176/dataset-master/JPEGImages/classes/LYMPHOCYTE'

def load_images():
    images = []
    for filename in os.listdir(data_dir):
        img = Image.open(os.path.join(data_dir, filename))
        img = img.convert('RGB') # Convert to RGB format
        img = img.resize((256, 256)) # Resize the image
        img = np.asarray(img)
        images.append(img)
    return np.asarray(images)


In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU, UpSampling2D, Activation

# Define the generator architecture
def build_generator(latent_dim, img_shape):
    model = Sequential()
    model.add(Dense(128 * int(img_shape[0] / 4) * int(img_shape[1] / 4), activation="relu", input_dim=latent_dim))
    model.add(Reshape((int(img_shape[0] / 4), int(img_shape[1] / 4), 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(img_shape[2], kernel_size=3, padding='same'))
    model.add(Activation("tanh"))
    return model



In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, ZeroPadding2D
from tensorflow.keras.models import Sequential

def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same'))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
# Define the loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Define the optimizer
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [ ]:

# Define the training loop
@tf.function
def train_step(images):
    # Generate random noise for the generator
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images using the generator
        generated_images = generator(noise, training=True)

        # Feed both real and fake images to the discriminator
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate the loss for both the generator and discriminator
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calculate the gradients and apply them to the generator and discriminator
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Define the training function
def train(dataset, epochs):
    for epoch in range(epochs):
        for batch in dataset:
            gen_loss, disc_loss = train_step(batch)

        print(f'Epoch {epoch+1}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}')


In [ ]:
# Set the hyperparameters
BATCH_SIZE = 32
LATENT_DIM = 100
EPOCHS = 700
img_shape = (256, 256, 3)
#discriminator = build_discriminator(img_shape)
# Load the dataset
images = load_images()

# Preprocess the images
images = (images.astype('float32') - 127.5) / 127.5
dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(len(images)).batch(BATCH_SIZE)

# Build the generator and discriminator
generator = build_generator(LATENT_DIM, img_shape)
discriminator = build_discriminator(img_shape)

# Define the loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Define the optimizer
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)




In [ ]:

def generator_loss(fake_output):
    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output))


In [ ]:

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
# Define the training loop
train(dataset, EPOCHS)

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Generator Loss: 1.8427708148956299, Discriminator Loss: 2.855480909347534
Epoch 2, Generator Loss: 1.6252357959747314, Discriminator Loss: 1.6725385189056396
Epoch 3, Generator Loss: 1.9210349321365356, Discriminator Loss: 0.7116492986679077
Epoch 4, Generator Loss: 0.9707484245300293, Discriminator Loss: 1.292386531829834
Epoch 5, Generator Loss: 1.2345104217529297, Discriminator Loss: 1.146935224533081
Epoch 6, Generator Loss: 1.4004091024398804, Discriminator Loss: 0.9380661845207214
Epoch 7, Generator Loss: 1.733908772468567, Discriminator Loss: 1.790191650390625
Epoch 8, Generator Loss: 1.1381916999816895, Discriminator Loss: 1.467679738998413
Epoch 9, Generator Loss: 0.39941126108169556, Discriminator Loss: 2.1669516563415527
Epoch 10, Generator Loss: 1.3694658279418945, Discriminator Loss: 1.6364755630493164
Epoch 11, Generator Loss: 1.282177448272705, Discriminator Loss: 2.653398275375366
Epoch 12, Generator Loss: 2.123133659362793, Discriminator Loss: 0.65999674797058

In [ ]:
from tensorflow.keras.models import load_model
generator_loaded = load_model('/content/drive/My Drive/generator_model.h5')

In [ ]:
# Generate 100 synthetic images using the trained generator
num_images = 13
latent_space_samples = tf.random.normal([num_images, LATENT_DIM])
generated_images = generator(latent_space_samples, training=False)

# Rescale the generated images to 0-255 range
generated_images = (generated_images * 127.5) + 127.5
generated_images = generated_images.numpy().astype(np.uint8)

# Save the generated images to disk
output_dir = '/content/drive/My Drive/synthetic_images/lymph'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(num_images):
    filename = os.path.join(output_dir, f'last13_{i+1}.jpg')
    img = Image.fromarray(generated_images[i])
    img.save(filename)
